In [1]:
import os
os.chdir('/<path-to-project>')
os.environ['PYTHONPATH'] = "/<path-to-project>"

In [2]:
%load_ext autoreload
%autoreload 2
from typing import List
import sys
sys.path.append('..')
from pathlib import Path
from string import Template
import pandas as pd
import numpy as np
from misc.analyses import load_results_table, create_results_table, load_results_table_for_dataset, prepare_dataset_results_df_for_paper
from approaches.ensemble_trainer import EnsembleTrainer

In [74]:
res_dirs = [
    'results_iclr/MOONS/processed/MOONS',
    'results_iclr/AMAZON_REVIEWS/processed/AMAZON_REVIEWS/',
    'results_iclr/MINI_DOMAIN_NET/processed/MINI_DOMAIN_NET/',
    'results_iclr/EEG/processed/EEG/',
    'results_iclr/HAR/processed/HAR/',
    'results_iclr/HHAR_SA/processed/HHAR_SA/',
    'results_iclr/WISDM/processed/WISDM/',
]

In [75]:
table_template_head = r"""\scalebox{0.51}{
\begin{tabular}{ l c | c c c c | c c c | c}
\toprule
 & & \multicolumn{4}{c|}{\textbf{Heuristic}} & \multicolumn{3}{c|}{\textbf{Theoretical error guarantees}}\\
 \cmidrule{2-10}
 \textbf{Dataset} & SO & TMV & TMR & TCR & SOR & IWV & DEV & IWA (ours) & TB \\
 \midrule
"""

table_template_mean = r"""
 {name}    & ${n1} (\pm {n2}$    & ${n3} (\pm {n4}$    & ${n5} (\pm {n6}$    & ${n7} (\pm {n8}$    & ${n9} (\pm {n10}$    & ${n11} (\pm {n12}$    & ${n13} (\pm {n14}$    & ${n15} (\pm {n16}$    & ${n17} (\pm {n18}$ \\
"""

table_template_end = r"""
 \bottomrule
\end{tabular}
}
\\~\\~\\"""
SO_ID = '0'
columns_in_order=[SO_ID, 'iwv', 'dev', 'source_reg', 'target_majority_reg', 'target_confidence_reg', 'target_majority_vote', 'agg', 'tb']

In [ ]:
# create latex code for tables
def name_mapping(name):
    if name in ['dann', 'cmd', 'mmd']:
        return name.upper()
    elif name == 'HHAR_SA':
        return 'HHAR'
    elif name == 'HAR':
        return 'UCI-HAR'
    elif name == 'EEG':
        return 'Sleep-EDF'
    elif name == 'MINI_DOMAIN_NET':
        return 'MiniDomainNet'
    elif name == 'AMAZON_REVIEWS':
        return 'Amazon Reviews'
    elif name == 'MOONS':
        return 'Transformed Moons'
    else:
        return name
def bf_(j):
    return r'{\bf ' if j == max_acc_idx else ''
def _bf(j):
    return r')}' if j == max_acc_idx else ')'
def it_(j):
    return r'{\it ' if j == heur_max_acc_idx else ''
def _it(j):
    return r')}' if j == heur_max_acc_idx else ')'

tex_table_str = table_template_head
for base_dir in res_dirs:
    total_mean = []
    dsr_df = load_results_table_for_dataset(base_dir)
    mean_df, std_df = prepare_dataset_results_df_for_paper(dsr_df, columns_in_order=columns_in_order)
    mean_row = '%s' % table_template_mean
    res_mean = mean_df.mean()
    res_std = std_df.mean()
    heur_max_acc = [0.0, res_mean['target_majority_vote'], res_mean['target_majority_reg'], res_mean['target_confidence_reg'], res_mean['source_reg'], 0.0, 0.0, 0.0, 0.0]
    heur_max_acc_idx = np.argmax(heur_max_acc)
    acc_list = [0.0, 0.0, 0.0, 0.0, 0.0, res_mean['iwv'], res_mean['dev'], res_mean['agg'], 0.0]
    acc_inv_list = acc_list[::-1]
    max_acc_idx = len(acc_inv_list) - np.argmax(acc_inv_list) - 1
    val = mean_row.format(
              name=name_mapping(Path(base_dir).stem).replace("_", "-"), \
              n1=bf_(0)+("%.3f" % np.around(res_mean[SO_ID], decimals=3)),                          n2=("%.3f" % np.around(res_std[SO_ID], decimals=3))+_bf(0), \
              n3=it_(1)+("%.3f" % np.around(res_mean['target_majority_vote'], decimals=3)),       n4=("%.3f" % np.around(res_std['target_majority_vote'], decimals=3))+_it(1), \
              n5=it_(2)+("%.3f" % np.around(res_mean['target_majority_reg'], decimals=3)),        n6=("%.3f" % np.around(res_std['target_majority_reg'], decimals=3))+_it(2), \
              n7=it_(3)+("%.3f" % np.around(res_mean['target_confidence_reg'], decimals=3)),      n8=("%.3f" % np.around(res_std['target_confidence_reg'], decimals=3))+_it(3), \
              n9=it_(4)+("%.3f" % np.around(res_mean['source_reg'], decimals=3)),                 n10=("%.3f" % np.around(res_std['source_reg'], decimals=3))+_it(4), \
              n11=bf_(5)+("%.3f" % np.around(res_mean['iwv'], decimals=3)),                       n12=("%.3f" % np.around(res_std['iwv'], decimals=3))+_bf(5), \
              n13=bf_(6)+("%.3f" % np.around(res_mean['dev'], decimals=3)),                       n14=("%.3f" % np.around(res_std['dev'], decimals=3))+_bf(6), \
              n15=bf_(7)+("%.3f" % np.around(res_mean['agg'], decimals=3)),                       n16=("%.3f" % np.around(res_std['agg'], decimals=3))+_bf(7), \
              n17=bf_(8)+("%.3f" % np.around(res_mean['tb'], decimals=3)),                        n18=("%.3f" % np.around(res_std['tb'], decimals=3))+_bf(8))
    tex_table_str += val
tex_table_str += table_template_end
print(tex_table_str)

In [78]:
table_template_head = r"""\scalebox{0.52}{
\begin{tabular}{ l c | c c c c | c c c | c}
\toprule
 & \multicolumn{9}{c}{\textbf{%s}}\\
 \cmidrule{2-10}
 & & \multicolumn{4}{c|}{\textbf{Heuristic}} & \multicolumn{3}{c|}{\textbf{Theoretical error guarantees}}\\
 \cmidrule{2-10}
 \textbf{Method} & SO & TMV & TMR & TCR & SOR & IWV & DEV & IWA (ours) & TB \\
 \midrule
"""

table_template_row = r"""
 {name}    & ${n1} (\pm {n2}$    & ${n3} (\pm {n4}$    & ${n5} (\pm {n6}$    & ${n7} (\pm {n8}$    & ${n9} (\pm {n10}$    & ${n11} (\pm {n12}$    & ${n13} (\pm {n14}$    & ${n15} (\pm {n16}$    & ${n17} (\pm {n18}$ \\
"""

table_template_mean = r"""
 Avg.      & ${n1} (\pm {n2}$    & ${n3} (\pm {n4}$    & ${n5} (\pm {n6}$    & ${n7} (\pm {n8}$    & ${n9} (\pm {n10}$    & ${n11} (\pm {n12}$    & ${n13} (\pm {n14}$    & ${n15} (\pm {n16}$    & ${n17} (\pm {n18}$ \\
"""

table_template_end = r"""
 \bottomrule
\end{tabular}
}
\\~\\~\\"""
SO_ID = '0'
columns_in_order=[SO_ID, 'iwv', 'dev', 'source_reg', 'target_majority_reg', 'target_confidence_reg', 'target_majority_vote', 'agg', 'tb']

In [ ]:
# create latex code for tables
def name_mapping(name):
    if name in ['dann', 'cmd', 'mmd']:
        return name.upper()
    elif name == 'HHAR_SA':
        return 'HHAR'
    elif name == 'HAR':
        return 'UCI-HAR'
    elif name == 'EEG':
        return 'Sleep-EDF'
    elif name == 'MINI_DOMAIN_NET':
        return 'MiniDomainNet'
    elif name == 'AMAZON_REVIEWS':
        return 'Amazon Reviews'
    elif name == 'MOONS':
        return 'Transformed Moons'
    else:
        return name
def bf_(j):
    return r'{\bf ' if j == max_acc_idx else ''
def _bf(j):
    return r')}' if j == max_acc_idx else ')'
def it_(j):
    return r'{\it ' if j == heur_max_acc_idx else ''
def _it(j):
    return r')}' if j == heur_max_acc_idx else ')'

for base_dir in res_dirs:
    total_mean = []
    dsr_df = load_results_table_for_dataset(base_dir)
    mean_df, std_df = prepare_dataset_results_df_for_paper(dsr_df, columns_in_order=columns_in_order)
    tex_table_str = table_template_head % name_mapping(Path(base_dir).stem).replace("_", "-")

    for (label, mean_row), (_, std_row) in zip(mean_df.iterrows(), std_df.iterrows()):
        heur_acc_list = [0.0, mean_row['target_majority_vote'], mean_row['target_majority_reg'], mean_row['target_confidence_reg'], mean_row['source_reg'], 0.0, 0.0, 0.0, 0.0]
        acc_list = [0.0, 0.0, 0.0, 0.0, 0.0, mean_row['iwv'], mean_row['dev'], mean_row['agg'], 0.0]
        total_mean.append([mean_row[SO_ID], mean_row['target_majority_vote'], mean_row['target_majority_reg'], mean_row['target_confidence_reg'], mean_row['source_reg'], mean_row['iwv'], mean_row['dev'], mean_row['agg'], mean_row['tb']])
        heur_max_acc_idx = np.argmax(heur_acc_list)
        acc_inv_list = acc_list[::-1]
        max_acc_idx = len(acc_inv_list) - np.argmax(acc_inv_list) - 1
        name_ = name_mapping(label)
        name_ = name_.replace("_", "-")
        val = table_template_row.format(name=name_,
                  n1=bf_(0)+("%.3f" % np.around(mean_row[SO_ID], decimals=3)),                        n2=("%.3f" % np.around(std_row[SO_ID], decimals=3))+_bf(0), \
                  n3=it_(1)+("%.3f" % np.around(mean_row['target_majority_vote'], decimals=3)),       n4=("%.3f" % np.around(std_row['target_majority_vote'], decimals=3))+_it(1), \
                  n5=it_(2)+("%.3f" % np.around(mean_row['target_majority_reg'], decimals=3)),        n6=("%.3f" % np.around(std_row['target_majority_reg'], decimals=3))+_it(2), \
                  n7=it_(3)+("%.3f" % np.around(mean_row['target_confidence_reg'], decimals=3)),      n8=("%.3f" % np.around(std_row['target_confidence_reg'], decimals=3))+_it(3), \
                  n9=it_(4)+("%.3f" % np.around(mean_row['source_reg'], decimals=3)),                 n10=("%.3f" % np.around(std_row['source_reg'], decimals=3))+_it(4), \
                  n11=bf_(5)+("%.3f" % np.around(mean_row['iwv'], decimals=3)),                       n12=("%.3f" % np.around(std_row['iwv'], decimals=3))+_bf(5), \
                  n13=bf_(6)+("%.3f" % np.around(mean_row['dev'], decimals=3)),                       n14=("%.3f" % np.around(std_row['dev'], decimals=3))+_bf(6), \
                  n15=bf_(7)+("%.3f" % np.around(mean_row['agg'], decimals=3)),                       n16=("%.3f" % np.around(std_row['agg'], decimals=3))+_bf(7), \
                  n17=bf_(8)+("%.3f" % np.around(mean_row['tb'], decimals=3)),                        n18=("%.3f" % np.around(std_row['tb'], decimals=3))+_bf(8))
        tex_table_str += val

    tex_table_str += ' \midrule'
    mean_row = '%s' % table_template_mean
    res_mean = mean_df.mean()
    res_std = std_df.mean()
    heur_max_acc = [0.0, res_mean['target_majority_vote'], res_mean['target_majority_reg'], res_mean['target_confidence_reg'], res_mean['source_reg'], 0.0, 0.0, 0.0, 0.0]
    heur_max_acc_idx = np.argmax(heur_max_acc)
    acc_list = [0.0, 0.0, 0.0, 0.0, 0.0, res_mean['iwv'], res_mean['dev'], res_mean['agg'], 0.0]
    acc_inv_list = acc_list[::-1]
    max_acc_idx = len(acc_inv_list) - np.argmax(acc_inv_list) - 1
    val = mean_row.format(
              n1=bf_(0)+("%.3f" % np.around(res_mean[SO_ID], decimals=3)),                          n2=("%.3f" % np.around(res_std[SO_ID], decimals=3))+_bf(0), \
              n3=it_(1)+("%.3f" % np.around(res_mean['target_majority_vote'], decimals=3)),       n4=("%.3f" % np.around(res_std['target_majority_vote'], decimals=3))+_it(1), \
              n5=it_(2)+("%.3f" % np.around(res_mean['target_majority_reg'], decimals=3)),        n6=("%.3f" % np.around(res_std['target_majority_reg'], decimals=3))+_it(2), \
              n7=it_(3)+("%.3f" % np.around(res_mean['target_confidence_reg'], decimals=3)),      n8=("%.3f" % np.around(res_std['target_confidence_reg'], decimals=3))+_it(3), \
              n9=it_(4)+("%.3f" % np.around(res_mean['source_reg'], decimals=3)),                 n10=("%.3f" % np.around(res_std['source_reg'], decimals=3))+_it(4), \
              n11=bf_(5)+("%.3f" % np.around(res_mean['iwv'], decimals=3)),                       n12=("%.3f" % np.around(res_std['iwv'], decimals=3))+_bf(5), \
              n13=bf_(6)+("%.3f" % np.around(res_mean['dev'], decimals=3)),                       n14=("%.3f" % np.around(res_std['dev'], decimals=3))+_bf(6), \
              n15=bf_(7)+("%.3f" % np.around(res_mean['agg'], decimals=3)),                       n16=("%.3f" % np.around(res_std['agg'], decimals=3))+_bf(7), \
              n17=bf_(8)+("%.3f" % np.around(res_mean['tb'], decimals=3)),                        n18=("%.3f" % np.around(res_std['tb'], decimals=3))+_bf(8))
    tex_table_str += val
    tex_table_str += table_template_end
    print(tex_table_str)

In [91]:
table_template_head = r"""\scalebox{0.55}{
\begin{tabular}{ l c | c c c c | c c c | c}
\toprule
 & \multicolumn{9}{c}{\textbf{%s}}\\
 \cmidrule{2-10}
 & & \multicolumn{4}{c|}{\textbf{Heuristic}} & \multicolumn{3}{c|}{\textbf{Theoretical error guarantees}}\\
 \cmidrule{2-10}
 \textbf{Task} & SO & TMV & TMR & TCR & SOR & IWV & DEV & IWA (ours) & TB \\
 \midrule
"""

table_template_row = r"""
 {name}    & ${n1} (\pm {n2}$    & ${n3} (\pm {n4}$    & ${n5} (\pm {n6}$    & ${n7} (\pm {n8}$    & ${n9} (\pm {n10}$    & ${n11} (\pm {n12}$    & ${n13} (\pm {n14}$    & ${n15} (\pm {n16}$    & ${n17} (\pm {n18}$ \\
"""

table_template_mean = r"""
 Avg.      & ${n1} (\pm {n2}$    & ${n3} (\pm {n4}$    & ${n5} (\pm {n6}$    & ${n7} (\pm {n8}$    & ${n9} (\pm {n10}$    & ${n11} (\pm {n12}$    & ${n13} (\pm {n14}$    & ${n15} (\pm {n16}$    & ${n17} (\pm {n18}$ \\
"""

table_template_end = r"""
 \bottomrule
\end{tabular}
}
\\~\\~\\"""

LOOKUP = {
    '0_src-11_tgt': r'0 $\rightarrow$ 11', 
    '12_src-5_tgt': r'12 $\rightarrow$ 5', 
    '16_src-1_tgt': r'16 $\rightarrow$ 1', 
    '7_src-18_tgt': r'7 $\rightarrow$ 18', 
    '9_src-14_tgt': r'9 $\rightarrow$ 14',
    '2_src-11_tgt': r'2 $\rightarrow$ 11', 
    '6_src-23_tgt': r'6 $\rightarrow$ 23', 
    '7_src-13_tgt': r'7 $\rightarrow$ 13', 
    '9_src-18_tgt': r'9 $\rightarrow$ 18', 
    '12_src-16_tgt': r'12 $\rightarrow$ 16',
    '0_src-6_tgt': r'0 $\rightarrow$ 6', 
    '1_src-6_tgt': r'1 $\rightarrow$ 6', 
    '2_src-7_tgt': r'2 $\rightarrow$ 7', 
    '3_src-8_tgt': r'3 $\rightarrow$ 8', 
    '4_src-5_tgt': r'4 $\rightarrow$ 5',
    '6_src-19_tgt': r'6 $\rightarrow$ 19', 
    '7_src-18_tgt': r'7 $\rightarrow$ 18', 
    '18_src-23_tgt': r'18 $\rightarrow$ 23', 
    '20_src-30_tgt': r'20 $\rightarrow$ 30', 
    '35_src-31_tgt': r'35 $\rightarrow$ 31',
    'books-dvd': r'B $\rightarrow$ D', 
    'books-electronics': r'B $\rightarrow$ E', 
    'books-kitchen': r'B $\rightarrow$ K', 
    'dvd-books': r'D $\rightarrow$ B', 
    'dvd-electronics': r'D $\rightarrow$ E', 
    'dvd-kitchen': r'D $\rightarrow$ K', 
    'electronics-books': r'E $\rightarrow$ B', 
    'electronics-dvd': r'E $\rightarrow$ D', 
    'electronics-kitchen': r'E $\rightarrow$ K', 
    'kitchen-books': r'K $\rightarrow$ B', 
    'kitchen-dvd': r'K $\rightarrow$ D', 
    'kitchen-electronics': r'K $\rightarrow$ E',
    'moons_src-moons_tgt': r'S $\rightarrow$ T',
    'books_src-dvd_tgt': r'B $\rightarrow$ D', 
    'books_src-electronics_tgt': r'B $\rightarrow$ E', 
    'books_src-kitchen_tgt': r'B $\rightarrow$ K', 
    'dvd_src-books_tgt': r'D $\rightarrow$ B', 
    'dvd_src-electronics_tgt': r'D $\rightarrow$ E', 
    'dvd_src-kitchen_tgt': r'D $\rightarrow$ K', 
    'electronics_src-books_tgt': r'E $\rightarrow$ B', 
    'electronics_src-dvd_tgt': r'E $\rightarrow$ D', 
    'electronics_src-kitchen_tgt': r'E $\rightarrow$ K', 
    'kitchen_src-books_tgt': r'K $\rightarrow$ B', 
    'kitchen_src-dvd_tgt': r'K $\rightarrow$ D', 
    'kitchen_src-electronics_tgt': r'K $\rightarrow$ E',
    '0_src-1_tgt': r'S $\rightarrow$ T',
    '1_src-0_tgt': r'T $\rightarrow$ S',
    'painting-real-sketch-clipart-infograph-quickdraw': r'CS $\rightarrow$ Q',
    'painting-quickdraw-sketch-clipart-infograph-real': r'CS $\rightarrow$ R',
    'painting-quickdraw-real-clipart-infograph-sketch': r'CS $\rightarrow$ S',
    'painting-quickdraw-real-sketch-infograph-clipart': r'CS $\rightarrow$ C',
    'painting-quickdraw-real-sketch-clipart-infograph': r'CS $\rightarrow$ I',
    'quickdraw-real-sketch-clipart-infograph-painting': r'CS $\rightarrow$ P',
    'real-sketch': r'R $\rightarrow$ S',
    'real-painting': r'R $\rightarrow$ P',
    'real-quickdraw': r'R $\rightarrow$ Q',
    'real-infograph': r'R $\rightarrow$ I',
    'real-clipart': r'R $\rightarrow$ C',
    'sketch-real': r'S $\rightarrow$ R',
    'sketch-painting': r'S $\rightarrow$ P',
    'sketch-quickdraw': r'S $\rightarrow$ Q',
    'sketch-infograph': r'S $\rightarrow$ I',
    'sketch-clipart': r'S $\rightarrow$ C',
    'quickdraw-real': r'Q $\rightarrow$ R',
    'quickdraw-painting': r'Q $\rightarrow$ P',
    'quickdraw-sketch': r'Q $\rightarrow$ S',
    'quickdraw-infograph': r'Q $\rightarrow$ I',
    'quickdraw-clipart': r'Q $\rightarrow$ C',
    'real_src-sketch_tgt': r'R $\rightarrow$ S',
    'real_src-painting_tgt': r'R $\rightarrow$ P',
    'real_src-quickdraw_tgt': r'R $\rightarrow$ Q',
    'real_src-infograph_tgt': r'R $\rightarrow$ I',
    'real_src-clipart_tgt': r'R $\rightarrow$ C',
}

In [102]:
#base_dir = 'results_iclr/AMAZON_REVIEWS/processed/AMAZON_REVIEWS/'
#base_dir = 'results_iclr/MINI_DOMAIN_NET/processed/MINI_DOMAIN_NET/'
#base_dir = 'results_iclr/EEG/processed/EEG/'
#base_dir = 'results_iclr/HAR/processed/HAR/'
#base_dir = 'results_iclr/HHAR_SA/processed/HHAR_SA/'
base_dir = 'results_iclr/WISDM/processed/WISDM/'

In [ ]:
def name_mapping(name):
    if name in ['dann', 'cmd', 'mmd']:
        return name.upper()
    elif name == 'HHAR_SA':
        return 'HHAR'
    elif name == 'HAR':
        return 'UCI-HAR'
    elif name == 'EEG':
        return 'Sleep-EDF'
    elif name == 'MINI_DOMAIN_NET':
        return 'MiniDomainNet'
    elif name == 'AMAZON_REVIEWS':
        return 'Amazon Reviews'
    elif name == 'MOONS':
        return 'Transformed Moons'
    else:
        return name

def bf_(j):
    return r'{\bf ' if j == max_acc_idx else ''
def _bf(j):
    return r')}' if j == max_acc_idx else ')'
def it_(j):
    return r'{\it ' if j == heur_max_acc_idx else ''
def _it(j):
    return r')}' if j == heur_max_acc_idx else ')'
    
# show mean accuracy tables
for e, da_method in enumerate(os.listdir(base_dir)):
    path = os.path.join(base_dir, da_method)
    src_tgt_acc_df = load_results_table(path)
    res_mean = src_tgt_acc_df.groupby(level=[0,2]).mean()
    res_std = src_tgt_acc_df.groupby(level=[0,2]).std()
    res_mean = res_mean[res_mean.index.get_level_values(1).str.contains('target')]
    res_std = res_std[res_std.index.get_level_values(1).str.contains('target')]
    header = '%s' % table_template_head
    mean_row = '%s' % table_template_mean
    da_method = name_mapping(da_method)
    res = header % da_method.replace("_", "-")
    tb_mean = []
    tb_std = []
    lambdas = []
    # filter out all single model columns for target best
    for c in res_mean.columns:
        try:
            _ = float(c)
            lambdas.append(c)
        except:
            pass
    for i, row in res_mean.iterrows():
        tb = row[lambdas].max()
        tb_mean.append(tb)
        tbidx = row.idxmax()
        tb_std.append(res_std[tbidx][i])
        row_temp = '%s' % table_template_row
        row_name = row.name[0]
        heur_max_acc = [0.0, res_mean['target_majority_reg'][i], res_mean['target_majority_reg'][i], res_mean['target_confidence_reg'][i], res_mean['source_reg'][i], 0.0, 0.0, 0.0, 0.0]
        heur_max_acc_idx = np.argmax(heur_max_acc)
        acc_list = [0.0, 0.0, 0.0, 0.0, 0.0, res_mean['iwv'][i], res_mean['dev'][i], res_mean['agg'][i], 0.0]
        acc_inv_list = acc_list[::-1]
        max_acc_idx = len(acc_inv_list) - np.argmax(acc_inv_list) - 1
        val = row_temp.format(name=LOOKUP[row_name], \
                  n1=bf_(0)+str(np.around(res_mean[SO_ID][i], decimals=3)),                      n2=str(np.around(res_std[SO_ID][i], decimals=3))+_bf(0), \
                  n3=it_(1)+str(np.around(res_mean['target_majority_reg'][i], decimals=3)),    n4=str(np.around(res_std['target_majority_reg'][i], decimals=3))+_it(1), \
                  n5=it_(2)+str(np.around(res_mean['target_majority_reg'][i], decimals=3)),    n6=str(np.around(res_std['target_majority_reg'][i], decimals=3))+_it(2), \
                  n7=it_(3)+str(np.around(res_mean['target_confidence_reg'][i], decimals=3)),  n8=str(np.around(res_std['target_confidence_reg'][i], decimals=3))+_it(3), \
                  n9=it_(4)+str(np.around(res_mean['source_reg'][i], decimals=3)),             n10=str(np.around(res_std['source_reg'][i], decimals=3))+_it(4), \
                  n11=bf_(5)+str(np.around(res_mean['iwv'][i], decimals=3)),                   n12=str(np.around(res_std['iwv'][i], decimals=3))+_bf(5), \
                  n13=bf_(6)+str(np.around(res_mean['dev'][i], decimals=3)),                   n14=str(np.around(res_std['dev'][i], decimals=3))+_bf(6), \
                  n15=bf_(7)+str(np.around(res_mean['agg'][i], decimals=3)),                   n16=str(np.around(res_std['agg'][i], decimals=3))+_bf(7), \
                  n17=bf_(8)+str(np.around(tb, decimals=3)),                                   n18=str(np.around(res_std[tbidx][i], decimals=3))+_bf(8))
        res += val
    res += ' \midrule'
    tb_mean = [float(s) for s in tb_mean]
    res_mean = src_tgt_acc_df.groupby('domain').mean()
    res_std = src_tgt_acc_df.groupby(level=[0,2]).std()#src_tgt_acc_df.groupby('domain').std()
    res_mean = res_mean[res_mean.index.get_level_values(0).str.contains('target')]
    res_std = res_std[res_std.index.get_level_values(1).str.contains('target')].groupby('domain').mean() #res_std = res_std[res_std.index.get_level_values(0).str.contains('target')]
    heur_max_acc = [0.0, res_mean['target_majority_reg'][0], res_mean['target_majority_reg'][0], res_mean['target_confidence_reg'][0], res_mean['source_reg'][0], 0.0, 0.0, 0.0, 0.0]
    heur_max_acc_idx = np.argmax(heur_max_acc)
    acc_list = [0.0, 0.0, 0.0, 0.0, 0.0, res_mean['iwv'][0], res_mean['dev'][0], res_mean['agg'][0], 0.0]
    acc_inv_list = acc_list[::-1]
    max_acc_idx = len(acc_inv_list) - np.argmax(acc_inv_list) - 1
    val = mean_row.format(
              n1=bf_(0)+str(np.around(res_mean[SO_ID][0], decimals=3)),                        n2=str(np.around(res_std[SO_ID][0], decimals=3))+_bf(0), \
              n3=it_(1)+str(np.around(res_mean['target_majority_reg'][0], decimals=3)),        n4=str(np.around(res_std['target_majority_reg'][0], decimals=3))+_it(1), \
              n5=it_(2)+str(np.around(res_mean['target_majority_reg'][0], decimals=3)),        n6=str(np.around(res_std['target_majority_reg'][0], decimals=3))+_it(2), \
              n7=it_(3)+str(np.around(res_mean['target_confidence_reg'][0], decimals=3)),      n8=str(np.around(res_std['target_confidence_reg'][0], decimals=3))+_it(3), \
              n9=it_(4)+str(np.around(res_mean['source_reg'][0], decimals=3)),                 n10=str(np.around(res_std['source_reg'][0], decimals=3))+_it(4), \
              n11=bf_(5)+str(np.around(res_mean['iwv'][0], decimals=3)),                       n12=str(np.around(res_std['iwv'][0], decimals=3))+_bf(5), \
              n13=bf_(6)+str(np.around(res_mean['dev'][0], decimals=3)),                       n14=str(np.around(res_std['dev'][0], decimals=3))+_bf(6), \
              n15=bf_(7)+str(np.around(res_mean['agg'][0], decimals=3)),                       n16=str(np.around(res_std['agg'][0], decimals=3))+_bf(7), \
              n17=bf_(8)+str(np.around(np.mean(tb_mean), decimals=3)),                         n18=str(np.around(np.mean(tb_std), decimals=3))+_bf(8))
    res += val
    res += table_template_end
    
    if 'AMAZON_REVIEWS' in base_dir:
        if e == 0: print('\n\n', '#'*50, 'PART 1', '#'*100, '\n\n')
        if e == 4: print('\n\n', '#'*50, 'PART 2', '#'*100, '\n\n')
        if e == 8: print('\n\n', '#'*50, 'PART 3', '#'*100, '\n\n')
    elif 'MINI_DOMAIN_NET' in base_dir or 'EEG' in base_dir or 'HAR' in base_dir or 'HHAR_SA' in base_dir or 'WISDM' in base_dir:
        if e == 0: print('\n\n', '#'*50, 'PART 1', '#'*100, '\n\n')
        if e == 6: print('\n\n', '#'*50, 'PART 2', '#'*100, '\n\n')
    
    print(res)
            